# 按令牌拆分

语言模型具有标记限制。您不应超过令牌限制。因此，当您将文本拆分为块时，最好计算标记的数量。有许多分词器。在计算文本中的标记时，应使用与语言模型中使用的相同的标记器。

## tiktoken
Tiktoken 是由 OpenAI 创建的快速 BPE 分词器。

我们可以用它来估计使用的token。对于 OpenAI 模型来说，它可能会更准确。

1. 文本的拆分方式：按传入的字符。
2. 如何测量块大小：通过 tiktoken 分词器。


In [1]:
# This is a long document we can split up.
with open("./files/liudehua.txt") as f:
    state_of_the_union = f.read()
from langchain_text_splitters import CharacterTextSplitter

该 .from_tiktoken_encoder() 方法要么 encoding 作为参数（例如 cl100k_base ），要么采用（ model_name 例如 gpt-4 ）。所有其他参数（如 chunk_size 、 chunk_overlap 和 separators ）都用于实例化 CharacterTextSplitter ：

In [5]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=5000, chunk_overlap=0
)
texts = text_splitter.split_text(state_of_the_union)

In [6]:
print(texts[0])

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。

1999年，刘德华获得“香港十大杰出青年”的荣誉，2000年11月则顺利荣登“世界十大杰出青年”[8]，成为获此殊荣的少数几位香港艺人。2006年7月7日，香港演艺学院因他“是香港最受尊重和喜爱的演艺名人之一，对香港电影及音乐贡献良多。其严谨专业的工作态度，足以成为年轻人的典范”，为了“表彰他在表演艺术方面的成就”而授予刘德华荣誉院士称号[9]，他也因此成为少数几位获此荣誉的香港艺人之一[10]。

刘德华笃信佛教，法号“慧果”，热心公益，时常参与慈善活动。2008年，刘德华获香港特别行政区政府委任为太平绅士[11]，2010年4月23日，刘德华获任中国残疾人福利基金会理事并担任副理事长[12]。2010年5月2日，刘德华获颁第十二届“世界杰出华人奖”同时获颁授加拿大纽奔驰域蓝仕桥大学荣誉博士学位[13]。2013年12月8日，他又当选香港残疾人奥委会暨伤残人士体育协会副会长[14]。2017年12月他因演艺事业和公益事务上的成就被香港树仁大学授予荣誉文学博士学位。2018年6月受邀加入美国电影艺术与科学学会成为会员[15]。


刘德华出生于香港新界大埔泰亨村（旧称菜坑村[注 1]）[16]，籍贯广东新会县荷塘镇（今江门市）[17]。祖父在当时算是大地主（乡村、农地）[18]。另外，他在家中亦有三姊、一妹和一弟（刘德盛）[19][20][21]，自己在家中排行第四[22]。其父刘礼年轻时为启德机场的消防员[16]。

刘德华五岁多时由于任职驻守机场消防员的父亲希望他能入读英文小学而随家人离开了农村[注 1]，全家后来搬到了九龙钻石山大磡村（曾就读村里的大磡村街坊福利会小学）[23]。钻石山为贫民区，多是木屋，容易发生火灾。刘家的木屋在刘德华十一岁时被大火烧毁，家人因此住在寮屋一年[18]，

请注意，如果我们使用 CharacterTextSplitter.from_tiktoken_encoder ，文本仅被拆分， CharacterTextSplitter tiktoken 而 tokenizer 用于合并拆分。这意味着拆分可以大于分 tiktoken 割器测量的块大小。我们可以用来 RecursiveCharacterTextSplitter.from_tiktoken_encoder 确保拆分不大于语言模型允许的标记块大小，如果每个拆分的大小较大，则每个拆分都将递归拆分：

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=500,
    chunk_overlap=0,
)

texts = text_splitter.split_text(state_of_the_union)
print(texts[0])

刘德华，BBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一[3]，也是吉尼斯世界纪录大全中获奖最多的香港歌手[4]；在影视方面，他三次获得香港电影金像奖最佳男主角奖，两次获得金马奖最佳男主角奖，至今参演电影超过170部[5]。刘德华是天幕公司和映艺集团的创建者，作为投资人与监制已参与制作了30多部华语电影[6]。除此之外，刘德华是四川省川剧学校客座教授[7]。


我们还可以直接加载一个 tiktoken 拆分器，这将确保每个拆分都小于块大小。

In [8]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

texts = text_splitter.split_text(state_of_the_union)
print(texts[0])

刘德华，B


一些书面语言（例如中文和日文）具有编码为 2 个或更多标记的字符。直接使用 可以在 TokenTextSplitter 两个块之间拆分字符的标记，从而导致格式错误的 Unicode 字符。使用 RecursiveCharacterTextSplitter.from_tiktoken_encoder 或 CharacterTextSplitter.from_tiktoken_encoder 确保块包含有效的 Unicode 字符串。

## spaCy

自然语言工具包，或更常见的 NLTK，是一套用 Python 编程语言编写的用于英语符号和统计自然语言处理 （NLP） 的库和程序。
我们不仅可以在“”上拆分， NLTK 还可以根据 NLTK 分词器进行拆分。
1. 如何拆分文本：通过 NLTK 分词器。
2. 如何测量块大小：按字符数。

In [1]:
# This is a long document we can split up.
with open("./files/liudehua.txt") as f:
    state_of_the_union = f.read()

In [2]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=1000)

In [ ]:
import nltk
# nltk.download('punkt')
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\thinkpad\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\thinkpad\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\thinkpad\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\thinkpad\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\thinkpad\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\thinkpad\

In [7]:
texts = text_splitter.split_text(state_of_the_union)
print(texts[0])

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\thinkpad/nltk_data'
    - 'D:\\anaconda3\\envs\\learn-text-splitters\\nltk_data'
    - 'D:\\anaconda3\\envs\\learn-text-splitters\\share\\nltk_data'
    - 'D:\\anaconda3\\envs\\learn-text-splitters\\lib\\nltk_data'
    - 'C:\\Users\\thinkpad\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


## KoNLPY

KoNLPy：Python 中的韩语 NLP 是一个用于韩语自然语言处理 （NLP） 的 Python 包。

标记拆分涉及将文本分割成更小、更易于管理的单元，称为标记。这些标记通常是单词、短语、符号或其他对进一步处理和分析至关重要的有意义的元素。在英语等语言中，标记拆分通常涉及用空格和标点符号分隔单词。代币拆分的有效性很大程度上取决于代币制作者对语言结构的理解，从而确保生成有意义的代币。由于为英语设计的分词器无法理解其他语言（如韩语）的独特语义结构，因此它们不能有效地用于韩语处理。

使用 KoNLPy 的 Kkma 分析器对韩语进行 token 拆分

在韩文文本的情况下，KoNLPY包括称为 Kkma （Korean Knowledge Morpheme Analyzer）的形态分析器。 Kkma 提供韩文文本的详细形态分析。它将句子分解为单词，将单词分解为各自的语素，识别每个标记的词性。它可以将文本块分割成单个句子，这对于处理长文本特别有用。

使用注意事项
虽然 Kkma 以其详细的分析而闻名，但需要注意的是，这种精度可能会影响处理速度。因此， Kkma 最适合分析深度优先于快速文本处理的应用。

In [ ]:
# This is a long Korean document that we want to split up into its component sentences.
with open("./your_korean_doc.txt") as f:
    korean_document = f.read()

In [ ]:
from langchain_text_splitters import KonlpyTextSplitter

text_splitter = KonlpyTextSplitter()

In [ ]:
texts = text_splitter.split_text(korean_document)
# The sentences are split with "\n\n" characters.
print(texts[0])

## Hugging Face tokenizer

Hugging Face 有许多标记器。
我们使用 Hugging Face 分词器，即 GPT2TokenizerFast 来计算以令牌为单位的文本长度。

1. 文本的拆分方式：按传入的字符。
2. 如何测量块大小：通过 Hugging Face 分词器计算的令牌数量。

In [ ]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
# This is a long document we can split up.
with open("./files/liudehua.txt") as f:
    state_of_the_union = f.read()
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
print(texts[0])